In [31]:
import pandas as pd
import zipcodes
import datetime
import gc
import numpy as np
import logging
today_str=str(datetime.datetime.now().date())

In [32]:
logging.basicConfig(filename='Saatva_Zip5_Plus4_'+today_str+'_run.log', level=logging.INFO)

In [33]:
EASI_1A=pd.read_csv("/home/jian/Docs/EASI/2018-07-19/ZIP4_18_DATA_A2_CSV/ZIP4_18_DATA_A2.CSV",
                    skiprows=1,usecols=['ZIP_CODE','ZIP4','HH18','MEDHHINC','PCTHH200P'],dtype=str)

In [34]:
EASI_1A['HH18']=EASI_1A['HH18'].astype(float)
EASI_1A['MEDHHINC']=EASI_1A['MEDHHINC'].astype(float)
EASI_1A['PCTHH200P']=EASI_1A['PCTHH200P'].astype(float)
EASI_1A=EASI_1A.rename(columns={"MEDHHINC":"Median_HH_Inc"})

In [35]:
DMA_Zips=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
DMA_Zips=DMA_Zips.iloc[:,[0,2,6]]
DMA_Zips.columns=['ZIP_CODE','DMA','state']

In [36]:
DMA_29=pd.read_csv("/home/jian/Projects/Saatva/High_Income_Zips_for_29_DMAs/Seleceted_DMA_Top25_TV25.csv",dtype=str)
DMA_29=DMA_29['dma'].unique().tolist()

In [37]:
DMA_Zips_29=DMA_Zips[DMA_Zips['DMA'].isin(DMA_29)]

In [38]:
DMA_Zips_29=DMA_Zips_29.drop_duplicates()

In [39]:
DMA_Zips_29=pd.merge(DMA_Zips_29,EASI_1A,on="ZIP_CODE",how="left")

In [40]:
len(DMA_Zips_29['ZIP_CODE'].unique())

14289

# On Median Income Zip+4

In [41]:
# Meidan Income $75K to $100K 
DMA_Zips_29_High=DMA_Zips_29[(DMA_Zips_29['Median_HH_Inc']<100000) & (DMA_Zips_29['Median_HH_Inc']>=75000)]

In [42]:
unique_zip_city=pd.DataFrame()
for zip_cd in DMA_Zips_29_High['ZIP_CODE'].unique().tolist():
    if len(zipcodes.matching(zip_cd))>=1:
        city=zipcodes.matching(zip_cd)[0]['city']
        df=pd.DataFrame({"ZIP_CODE":zip_cd,"city":city},index=[zip_cd])
        unique_zip_city=unique_zip_city.append(df)
    else:
        print(zip_cd)
        logging.info(str(zip_cd) + ": no city in zipcodes packages, google for it")

63380
86005
97003


In [43]:
unique_zip_city=unique_zip_city.append(pd.DataFrame({"ZIP_CODE":['63380','86005','97003'],"city":["Truesdale","Kachina Village","Hillsboro"]},index=['63380','86005','97003']))



In [44]:
DMA_Zips_29_High=pd.merge(DMA_Zips_29_High,unique_zip_city,on="ZIP_CODE",how="left")

In [45]:
DMA_Zips_29_High=DMA_Zips_29_High[['ZIP_CODE','ZIP4','DMA','city','state','HH18','Median_HH_Inc','PCTHH200P']]

In [46]:
DMA_Zips_29_High['HH_200K_Plus']=DMA_Zips_29_High['HH18']/100*DMA_Zips_29_High['PCTHH200P']

In [ ]:
writer=pd.ExcelWriter("/home/jian/Projects/Saatva/High_Income_Zips_for_29_DMAs/Saatva_Zip Plus4 Median HH Inc 75K to 100K for 29 DMAs_JL_20180823.xlsx",engine="xlsxwriter")

In [18]:
for dma in DMA_29:
    df=DMA_Zips_29_High[DMA_Zips_29_High['DMA']==dma]
    df=df.reset_index()
    del df['index']
    if df.shape[0]>1000000:
        df=df.sort_values('Avg_HH_Inc',ascending=False)
        df=df.reset_index()
        del df['index']
        df_0=df.iloc[0:50000,]
        df_1=df.iloc[50000:,]
        df_0.to_excel(writer,dma+"_1",index=False)
        df_1.to_excel(writer,dma+"_2",index=False)
    else:
        df.to_excel(writer,dma,index=False)
    print(dma+" with rows of: "+str(df.shape[0]))

NEW YORK with rows of: 331140
LOS ANGELES with rows of: 318610
SAN FRANCISCO-OAK-SAN JOSE with rows of: 127572
WASHINGTON, DC (HAGRSTWN) with rows of: 139201
CHICAGO with rows of: 257954
BOSTON (MANCHESTER) with rows of: 192105
SEATTLE-TACOMA with rows of: 171818
PHILADELPHIA with rows of: 202422
DALLAS-FT. WORTH with rows of: 206987
DENVER with rows of: 109803
ATLANTA with rows of: 180183
PHOENIX (PRESCOTT) with rows of: 110595
TAMPA-ST. PETE (SARASOTA) with rows of: 93671
HOUSTON with rows of: 160064
BALTIMORE with rows of: 72930
ORLANDO-DAYTONA BCH-MELBRN with rows of: 81707
MINNEAPOLIS-ST. PAUL with rows of: 178281
DETROIT with rows of: 133099
MIAMI-FT. LAUDERDALE with rows of: 83558
SAN DIEGO with rows of: 57048
SACRAMNTO-STKTON-MODESTO with rows of: 85691
AUSTIN with rows of: 68873
PORTLAND, OR with rows of: 125018
WEST PALM BEACH-FT. PIERCE with rows of: 45184
RALEIGH-DURHAM (FAYETVLLE) with rows of: 99175
CHARLOTTE with rows of: 70863
CLEVELAND-AKRON (CANTON) with rows of: 1037

In [19]:
writer.save()

In [21]:
gc.collect()

0

# On Median Income Zip5

In [47]:
del EASI_1A
del DMA_Zips_29_High
del df

In [48]:
zip5_HH=DMA_Zips_29.groupby(['ZIP_CODE','DMA','state'])['HH18'].sum().to_frame().reset_index()

In [49]:
DMA_Zips_29.shape

(18382899, 7)

In [50]:
for col in DMA_Zips_29.columns.tolist():
    print(col, DMA_Zips_29[col].apply(lambda x: type(x)).unique().tolist())


ZIP_CODE [<class 'str'>]
DMA [<class 'str'>]
state [<class 'str'>]
ZIP4 [<class 'str'>, <class 'float'>]
HH18 [<class 'float'>]
Median_HH_Inc [<class 'float'>]
PCTHH200P [<class 'float'>]


In [51]:
temp_DMA_Zip_29=DMA_Zips_29[['ZIP_CODE','ZIP4','HH18','Median_HH_Inc','PCTHH200P']].drop_duplicates(['ZIP_CODE','ZIP4'])

In [52]:
del temp_DMA_Zip_29['ZIP4']
gc.collect()

195

In [53]:
temp_DMA_Zip_29['Median_HH_Inc'].max()

650328.0

In [54]:
i=0
zip5_HH_Median_Inc=pd.DataFrame()

for zip_cd,group in temp_DMA_Zip_29.groupby(['ZIP_CODE']):
    
    total_HH=group['HH18'].sum()
    group=group[['ZIP_CODE','HH18','Median_HH_Inc','PCTHH200P']]
    
    
    agg_Pctg_HH_200P=sum(group['PCTHH200P']*group['HH18']/total_HH)
    agg_median=sum(group['Median_HH_Inc']*group['HH18']/total_HH)
    
    df_app=pd.DataFrame({"ZIP_CODE":zip_cd,"Meidan_HH_Inc_agg":agg_median,"Pctg_HH_200P_agg":agg_Pctg_HH_200P},index=[zip_cd])
    
    zip5_HH_Median_Inc=zip5_HH_Median_Inc.append(df_app)
    
    i=i+1
    if i % 1000==10:
        print(i,datetime.datetime.now())
        logging.info(str(i)+"|"+str(datetime.datetime.now()))
        gc.collect()
    if i % 1000==100:
        print(i,datetime.datetime.now())
        logging.info(str(i)+"|"+str(datetime.datetime.now()))
    

10 2018-08-29 13:44:54.187378
100 2018-08-29 13:44:54.525498
1010 2018-08-29 13:44:57.704691
1100 2018-08-29 13:44:58.059080
2010 2018-08-29 13:45:01.335190
2100 2018-08-29 13:45:01.699831
3010 2018-08-29 13:45:04.911868
3100 2018-08-29 13:45:05.261486
4010 2018-08-29 13:45:08.499946
4100 2018-08-29 13:45:08.860273
5010 2018-08-29 13:45:12.227052
5100 2018-08-29 13:45:12.605461
6010 2018-08-29 13:45:16.107537
6100 2018-08-29 13:45:16.490507
7010 2018-08-29 13:45:20.035434
7100 2018-08-29 13:45:20.409700
8010 2018-08-29 13:45:23.983453
8100 2018-08-29 13:45:24.353719
9010 2018-08-29 13:45:27.971788
9100 2018-08-29 13:45:28.361966
10010 2018-08-29 13:45:32.072054
10100 2018-08-29 13:45:32.480778
11010 2018-08-29 13:45:36.186890
11100 2018-08-29 13:45:36.571456
12010 2018-08-29 13:45:40.380379
12100 2018-08-29 13:45:40.788922
13010 2018-08-29 13:45:44.581456
13100 2018-08-29 13:45:44.986634
14010 2018-08-29 13:45:48.792097
14100 2018-08-29 13:45:49.214859


In [55]:
zip5_HH_Median_Inc_High[zip5_HH_Median_Inc_High['Meidan_HH_Inc_agg']==100000]
# Check to confirm empty

,ZIP_CODE,DMA,city,state,HH18,Meidan_HH_Inc_agg,Pctg_HH_200P_agg,HH_200K_Plus


In [25]:
# Zip 5 level HH Inc $75K to $100K
zip5_HH_Median_Inc_High=zip5_HH_Median_Inc[(zip5_HH_Median_Inc['Meidan_HH_Inc_agg']<100000) & (zip5_HH_Median_Inc['Meidan_HH_Inc_agg']>=75000)]
zip5_HH_Median_Inc_High=pd.merge(zip5_HH_Median_Inc_High,zip5_HH,on="ZIP_CODE",how="left")
zip5_HH_Median_Inc_High['HH_200K_Plus']=zip5_HH_Median_Inc_High['HH18']*zip5_HH_Median_Inc_High['Pctg_HH_200P_agg']/100

In [26]:
unique_zip_city=pd.DataFrame()
for zip_cd in zip5_HH_Median_Inc_High['ZIP_CODE'].unique().tolist():
    if len(zipcodes.matching(zip_cd))>=1:
        city=zipcodes.matching(zip_cd)[0]['city']
        df=pd.DataFrame({"ZIP_CODE":zip_cd,"city":city},index=[zip_cd])
        unique_zip_city=unique_zip_city.append(df)
    else:
        print(zip_cd)
# No need to fill through google

86005


In [22]:
# Stopped here

In [27]:
zip5_HH_Median_Inc_High=pd.merge(zip5_HH_Median_Inc_High,unique_zip_city,on="ZIP_CODE",how="left")

In [28]:
zip5_HH_Median_Inc_High=zip5_HH_Median_Inc_High[['ZIP_CODE','DMA','city','state','HH18','Meidan_HH_Inc_agg','Pctg_HH_200P_agg','HH_200K_Plus']]

In [29]:
writer=pd.ExcelWriter("/home/jian/Projects/Saatva/High_Income_Zips_for_29_DMAs/Saatva_Zip5 Median HH Inc 75K to 100K for 29 DMAs_JL_20180823.xlsx",engine="xlsxwriter")
zip5_HH_Median_Inc_High.to_excel(writer,"all_29_DMAs",index=False)

for dma in DMA_29:
    df=zip5_HH_Median_Inc_High[zip5_HH_Median_Inc_High['DMA']==dma]
    df.to_excel(writer,dma,index=False)


In [30]:
writer.save()